In [6]:
# Reset the schema
import duckdb

# Connect to your DuckDB database (file or in-memory)
conn = duckdb.connect("mtga_local.duckdb")  # or ':memory:'

# Read the SQL file
with open("silver_duckdb_schema.sql", "r") as f:
    sql_script = f.read()

# Execute the entire SQL script
conn.execute(sql_script)

# Optional: check tables
tables = conn.execute("SHOW TABLES").fetchall()
conn.close()
print(tables)


[('decks',), ('dim_cards',), ('matches',), ('players',), ('turn1_hands',)]


In [ ]:
# Load the dim_cards table
import pandas as pd
import duckdb
import json

# 1. Read CSV safely
df = pd.read_csv(
    '/home/r3gal/develop/mtga_pipeline/data/references/dim_cards.csv',
    dtype=str,      # read all columns as strings
    keep_default_na=False  # keep empty fields as empty strings instead of NaN if desired
)

# Optional: Convert JSON/list columns from strings to JSON strings
json_cols = ['colors', 'color_identity', 'legalities']
for col in json_cols:
    df[col] = df[col].apply(lambda x: x if x.startswith('[') or x.startswith('{') else '[]')

df = df.drop_duplicates(keep='first', subset='arena_id')
# 2. Connect to DuckDB
conn = duckdb.connect("mtga_local.duckdb")

# 3. Register the dataframe and insert into dim_cards
conn.register("df_temp", df)
conn.execute("INSERT INTO dim_cards SELECT * FROM df_temp")

# 4. Preview
# print(conn.execute("SELECT * FROM dim_cards LIMIT 5").fetchall())
conn.close()


[('91829', 'b34bb2dc-c1af-4d77-b0b3-a0fb342a5fc6', 'Forest', datetime.date(2024, 8, 2), 'https://scryfall.com/card/blb/280/forest', '', 0, '[]', "['G']", 'Basic Land — Forest', 'blb', 'Bloomburrow', 'expansion', 'common', "{'standard': 'legal', 'future': 'legal', 'historic': 'legal', 'timeless': 'legal', 'gladiator': 'legal', 'pioneer': 'legal', 'modern': 'legal', 'legacy': 'legal', 'pauper': 'legal', 'vintage': 'legal', 'penny': 'legal', 'commander': 'legal', 'oathbreaker': 'legal', 'standardbrawl': 'legal', 'brawl': 'legal', 'alchemy': 'legal', 'paupercommander': 'legal', 'duel': 'legal', 'oldschool': 'not_legal', 'premodern': 'legal', 'predh': 'legal'}"), ('66119', '9f0d82ae-38bf-45d8-8cda-982b6ead1d72', 'Siren Lookout', datetime.date(2017, 9, 29), 'https://scryfall.com/card/xln/78/siren-lookout', '{2}{U}', 3, "['U']", "['U']", 'Creature — Siren Pirate', 'xln', 'Ixalan', 'expansion', 'common', "{'standard': 'not_legal', 'future': 'not_legal', 'historic': 'legal', 'timeless': 'legal'

In [3]:

con.close()

In [ ]:
# insert matches
CREATE TABLE matches (
    match_id BIGINT PRIMARY KEY,
    deck_id BIGINT NOT NULL,
    player_id VARCHAR,
    player_seat INTEGER, -- 1 or 2
    start_time TIMESTAMP NOT NULL,
    duration INTEGER,               -- SECONDS
    winner_seat VARCHAR,  -- 1 or 2
    format VARCHAR,
    draw_order VARCHAR
);



# intake
match_id, deck_id, 

# cehck for deck_id match??

#  Big challenge -> getting draw order



In [ ]:
# returns the deck_id, of the new deck or matching old deck
def insert_deck (conn, df, player_id, match_id):

    # check for a matching deck
    # if found return the deck_id

    # else find the most recent deck_id and add the new deck
    # return the new deck_id

    deck_obj = df.iloc[0]['payload'].get('request') 
    nested = json.loads(deck_obj)

    # Gets the array of the main deck: cardId and quantity are keys
    # [{'cardId': 95192, 'quantity': 9},
    #  {'cardId': 93715, 'quantity': 1},
    #  {'cardId': 95200, 'quantity': 6}]

    deck_name = nested.get('Summary').get('Name')
    deck_list = nested.get('Deck').get('MainDeck')
    deck_sideboard = nested.get('Deck').get('Sideboard')
    deck_commander = nested.get('Deck').get('CommandZone')

    # convert to string for insertion
    deck_list_json = json.dumps(deck_list) if deck_list else None
    deck_sideboard_json = json.dumps(deck_sideboard) if deck_sideboard else None
    deck_commander_json = json.dumps(deck_commander) if deck_commander else None

#   checking if the deck already exists, returning the deck_id if it does
    old_deck = conn.execute("""
        SELECT deck_id from decks 
        WHERE deck_name = ?
        and deck_list = ?
        and deck_sideboard = ?
        and deck_commander = ?
        """,
        (deck_name, deck_list_json, deck_sideboard_json, deck_commander_json)
        ).fetchone()
    if len(old_deck) > 0:
        return old_deck[0]

#   else getting the most recent deck_id and inserting the new deck
    row = conn.execute("SELECT COALESCE(MAX(deck_id), 0) FROM decks").fetchone()
    next_deck_id = row[0] + 1

    conn.execute(
        "INSERT INTO decks (deck_id, player_id, match_id, deck_list, deck_sideboard, deck_commander) VALUES (?, ?, ?, ?, ?, ?)",
        (next_deck_id, player_id, match_id, deck_list, deck_sideboard, deck_commander)
        )
    return next_deck_id